In [ ]:
import numpy as np 
from keras.applications.inception_v3 import InceptionV3 
from keras.models import Sequential, load_model, Model 
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, GlobalAveragePooling2D 
from keras.optimizers import SGD 
from keras.utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import TensorBoard 
import itertools

In [ ]:
ROWS = 256
COLS = 256
CHANNELS = 3

In [ ]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45) 
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('CUB_200_2011/train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('CUB_200_2011/test', target_size=(ROWS, COLS), class_mode='categorical') 

In [ ]:
#create the base pre-trained model 
base_model = InceptionV3(weights='imagenet', include_top=False) 

#add a global spatial average pooling Layer 
x = base_model.output 
x = GlobalAveragePooling2D()(x) 

#add a fully-connected Layer 
x = Dense (1024, activation='relu')(x) 
out_layer = Dense(200, activation='softmax')(x) 

# this is the model we will train 
model = Model(inputs=base_model.input, outputs=out_layer)

In [ ]:
#first: train only the top Loyers (which were randomly initialized) 
#i.e. freeze all convolutional InceptionV3 Layers 

for layer in base_model.layers: 
    layer.trainable = False 

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
model.summary()

In [ ]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='./logs')

model.fit(train_generator, steps_per_epoch=32, epochs = 100, callbacks=[tensorboard], verbose=2)

In [ ]:
print(model.evaluate(test_generator, steps=5000))

In [ ]:
#unfreeze all layers for more training. 
for layer in model.layers: 
    layer.trainable = True 
    
# we need to recompile the model for these modifications to take effect 
# we use SGD with a Low Learning rate 
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss="categorical_crossentropy", metrics=["accuracy"]) 

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100)

In [ ]:
test_generator.reset()
print(model.evaluate(test_generator, steps=5000))

In [ ]:
model.save("birds-inceptionv3.keras")